In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')
warnings.filterwarnings('ignore', category=DeprecationWarning, module='pyLDAvis')

import pyLDAvis
import pyLDAvis.sklearn

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [2]:
def get_top_words(model, feature_names, n_top_words):
    top_words = {}
    for topic_idx, topic in enumerate(model.components_):
        _top_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        top_words[str(topic_idx)] = _top_words
    return(top_words)

In [3]:
pyLDAvis.enable_notebook()

In [193]:
df = pd.read_pickle('../data/df_new.pkl')
df = df.drop_duplicates(['description', 'url'])
text = df['description'].values.tolist()
max_features = 5000
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=3,
                                max_features=max_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(text)
print("ready")

ready


In [194]:
#n_topics = 5 and max_features = 5000 creates fully separated topics
#n_topics = 10 and max_features = 2000 creates relatively good separation
#n_topics = 12 and max_features = 3000m mds='tsne' gives perfect separation
n_topics = 10
lda_model = LatentDirichletAllocation(n_components=n_topics, max_iter=5,
                                      learning_method='online',
                                      learning_offset=50.,
                                      random_state=0)

lda_model.fit(tf)
article_data = pyLDAvis.sklearn.prepare(lda_model,tf, tf_vectorizer, R=20, mds='tsne')

/Users/oliviacarnes/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [195]:
pyLDAvis.display(article_data)

In [196]:
df_topics = pd.DataFrame(lda_model.transform(tf))
df = df.join(df_topics)

In [197]:
value_of_interest = 7
# print('Topic: '+topic_names[value_of_interest])
df.sort_values([value_of_interest], ascending=False).head(10)[['price','url','description']].values.tolist()

[[915,
  'https://www.apartments.com/1325-corona-st-denver-co-unit-10/zmb978x/',
  'bedroom apartment charm corona_st victoria building central location near_downtown cheeseman_park grocery store king soopers whole_foods restaurant bar music venue floor ceiling_fan clawfoot_tub large window available june_1_2017 parking available rent include gas electricity amelia set time apartment showing appointment'],
 [650,
  'https://www.apartments.com/1020-15th-st-denver-co/xyclpzm/',
  'bed2 bath condo fully furnish heart downtown love hardwood floor floor ceiling window fireplace porch overlooking granite countertop expansive master suite make loft comfortable live washer dryer unit dishwasher dryer freezer garbage disposal microwave_range_ oven refrigerator washer deed underground garage spot include kindy contact direct home address thank rlne3830242 amenities furnished parking garage appliance dishwasher refrigerator oven garbage disposal washer__dryer microwave lease length month month re

In [ ]:
7